<a href="https://colab.research.google.com/github/viti990/my_own_llama/blob/main/llamav27b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aviator LLaMA

This notebook aims at reconstructing LLAMA 2 7B architecture and use the model weights trained from meta to learn how it is done!

Also the aim is to adapt the model with LoRA and QLoRA for PEFT!

The fine tuning will be done with aviation regulations from 14 CFR...


## 1.0 Downloading the weights

Downloading the weights from meta for 7B-chat (i want the model to be able to respond to questions from the requirements)
for this one must go to the meta website and request access, then go the github and use the download.sh scrip as shown below.


In [ ]:
!git clone https://github.com/meta-llama/llama/
!mv ./llama/download.sh ./
!rm -rf llama
!bash download.sh
!rm -rf LICENSE USE_POLICY.md tokenizer_checklist.chk

Cloning into 'llama'...
remote: Enumerating objects: 464, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 464 (delta 17), reused 33 (delta 12), pack-reused 417
Receiving objects: 100% (464/464), 1.12 MiB | 24.30 MiB/s, done.
Resolving deltas: 100% (235/235), done.
Enter the URL from email: https://download.llamameta.net/*?Policy=eyJTdGF0ZW1lbnQiOlt7InVuaXF1ZV9oYXNoIjoidGQxOHdlcHp2M21ubGl6YzJqZTQ0Nm40IiwiUmVzb3VyY2UiOiJodHRwczpcL1wvZG93bmxvYWQubGxhbWFtZXRhLm5ldFwvKiIsIkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMDE5ODM0OX19fV19&Signature=qbEw3yZ-aOH%7ElMJPCaqdLdxv6cFeKlza-RFTWxnd54SEePs6awC85oYIB1Qr9pk6dSp6kpBcBJRwtzJSEN0dj-WPmvUqVyZU4M3q2A-qakJUhO3hoo8lhroBxTYHOVCc0n8IQTMNLSkYsBzpu7Sfdeizkq%7EkBzCB140NNChA4QcjWgaF1P-KWPqOLAtgC1b1VPHo25qB1sbZTrOix3wmVatrHg79RjsjHbqIpL%7ECmZhBtpkauJG4JG2FMZNiPCH3NzG8am2rociEogyIFoeZTKBTKeE7NdOOHCNcFe6tanQMo7FmRXMWwnmeRW5ShwpU6fmfp1qaJH4iFYcQrVf-Kw__&Key-Pair-Id=K15QRJLY

## 2. Creating the architecture

### 2.1 Importing everything...

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
from typing import Optional

In [ ]:
@dataclass
class ModelArgs:
  dim: int=4096
  n_layers: int=32
  n_heads: int=32 #number of heads for the queries
  n_kv_heads: Optional[int]=None #number of values for the key and value
  vocab_size: int = -1 # This will be set when we load the tokenizer
  multiple_of: int = 256
  ffn_dim_multiplier: Optional[float]=None
  norm_eps: float = 1e-5

  #Needed for KV cache
  max_batch_size: int=32
  max_seq_len: int=2048

  device: str=None

def precompute_theta_pos_frequencies(head_dim: int, seq_len: int, device: str, theta: float = 10000.0 ):
  # as written in the paper, the dimension of the embedding must be even.
  assert head_dim % 2 ==0, "Dimension must be divisible by 2"
  # Build the theta parameters
  # According to the formula theta_i = 10000^ (-2(i-1)/dim) for i = [1, 2, ... dim / 2]
  # Shape: (head_dim / 2)
  theta_numerator = torch.arange(0, head_dim, 2).float()
  theta = 1.0 / (theta ** (theta_numerator/head_dim)).to(device)
  # Construct the positions (the "m" parameter)
  # shape: (seq_len)
  m = torch.arange(seq_len, device=device)
  # Multiply each theta by each position using the outer product
  # Shape: {Seq_Len} outer_product * (head_dim / 2) -> (seq_len, head_dim / 2)
  freqs = torch.outer(m, theta).float()
  # we can compute complex numbers in the polar form c = R * exp(i * m * theta), where r = 1 as follows:
  freqs_complex = torch.polar(torch.ones_like(freqs), freqs)

  return freqs_complex

def apply_rotary_embedding(x: torch.Tensor, freqs_complex: torch.Tensor, device: str):
  # (B, Seq_len, H, Head_dim) -> (B,Seq_len, H, head_dim/2)
  x_complex = torch.view_as_complex(x.float().reshape(*x.shape[:1], -1, 2))
  # (Seq_len, head_dim / 2) -> (1, Seq_len, head_dim / 2)
  freqs_complex = freqs_complex.unsqueeze(0).unsqueeze(2)
  # (B, Seq_len, H, head_dim / 2) * (1, Seq_len, 1, head_dim / 2) * (B, seq_len, H, Head_Dim / 2)
  x_rotated = x_complex * freqs_complex
  # (B, seq_len, H, head_dim / 2) -> (B, seq_len, H, head_dim / 2, 2)
  x_out = torch.view_as_real(x_rotated)
  #(B, seq_len, H, head_dim / 2, 2) -> (B, seq_len, H, head_dim)
  x_out = x_out.reshape(*x.shape)

  return x_out.type_as(x).to(device)

def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
  batch_size, seq_len, n_kv_heads = x.shape
  if n_rep == 1:
     return x
  else:
    return (
        x[:, :, :, None, :]
        .expand(batch_size,seq_len, n_kv_heads, n_rep, head_dim)
        .reshape(batch_size, seq_len, n_kv_heads * n_rep, head_dim)
    )

class RMSNorm(nn.Module):
  def __init__(self, dim: int, eps: float=1e-5) -> None:
    super().__init__()
    self.eps = eps
    # the gamma parameter
    self.weight = nn.Parameter(torch.ones(dim))

  def _norm(self, x: torch.Tensor) -> torch.Tensor:
    # (B, seq_len, dim)
    # rsqrt: 1/sqrt(x)
    return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True)) + self.eps

  def forward(self, x: torch.Tensor):
    #(dim) * (B, Seq_len, dim) * (B, seq_len, dim)
    return self.weight * self._norm(x.float()).type_as(x)

class SelfAttention(nn.Module):
  def __init__(self, args: ModelArgs):
    super().__init__()
    # Indicates the number of heads fot the key and values
    self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
    # Indicates the number of heads for the queries
    self.n_heads_q = args.n_heads
    # Indicates how many times the heads of keys and values should be repeated to match the head of the queries
    self.n_rep = self.n_heads_q // self.n_kv_heads
    # Indicates the dimension of each head
    self.head_dim = args.dim // args.n_heads

    self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias = False)
    self.wk = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias = False)
    self.wv = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias = False)
    self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim,  bias = False)

    self.cache_k = torch.zeros((args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim))
    self.cache_v = torch.zeros((args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim))

  def forward(self, x: torch.Tensor, start_pos: int, freqs_complex: torch.Tensor):
    batch_size, seq_len, _ = x.shape #(B, 1, dim)

    # Apply the wq, wk, wv matrices to queries, keys and values
    # (B, 1, dim) -> (B, 1, H_Q * head_dim)
    xq = self.wq(x)

    # (B, 1, dim) -> (B, 1, H_KV * head_dim)
    xk = self.wk(x)
    xv = self.wv(x)

    # (B, 1, H_Q * head_dim) -> (B, 1, H_Q, head_dim)
    xq = xq.view(batch_size, 1, self.n_heads_q, self.head_dim)

    # (B, 1, H_KV * head_dim) -> (B, 1, H_KV, head_dim)
    xk = xk.view(batch_size, 1, self.n_kv_heads, self.head_dim)
    xv = xv.view(batch_size, 1, self.n_kv_heads, self.head_dim)

    # Does not change the shape of the tensors
    xq = apply_rotary_embedding(xq, freqs_complex, device = x.device)
    xk = apply_rotary_embedding(xk, freqs_complex, device = x.device)

    # Replace the entry in the cache fot this token
    self.cache_k[:batch_size, start_pos:start_pos+seq_len] = xk
    self.cache_v[:batch_size, start_pos:start_pos+seq_len] = xv

    # Retrieve all the cached keys and values so far
    # (B, seq_len_KV, H_KV, head_dim)
    keys = self.cache_k[:batch_size, 0:start_pos+seq_len]
    values = self.cache_v[:batch_size, 0:start_pos+seq_len]

    # Repeat the heads of the K and V to reach the number of heads of the queries
    keys = repeat_kv(keys, self.n_rep)
    values = repeat_kv(values, self.n_rep)

    # (B, 1, H_Q, head_dim) -> (B, H_Q, 1, head_dim)
    xq = xq.transpose(1, 2)
    keys = keys.transpose(1, 2)
    values = values.transpose(1, 2)

    # (B, H_Q, 1, seq_len) @ (B, H_Q, seq_len_kv, head_dim) -> (B, H_Q, 1, head_dim)
    scores = torch.matmul(xq, keys.transpose(2, 3)) / math.sqrt(self.head_dim)
    scores = F.softmax(scores, dim=-1).type_as(xq)

    # (B, H_Q, 1, head_dim) -> (B, 1, H_Q, head_dim) -> (B, 1, dim)
    output = torch.matmul(scores, values)
    output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, -1)

    return self.wo(output)

class FeedForward(nn.Module):
  def __init__(self, args: ModelArgs):
    super().__init__()

    hidden_dim = 4 * args.dim
    hidden_dim = int(2 * hidden_dim / 3)
    if args.ffn_dim_multiplier is not None:
      hidden_dim = int(args.ffn_dim_multiplier * hidden_dim)
    # round the hidden_dim to the nearest multiple of the multiple_of parameter
    hidden_dim = args.multiple_of * ((hidden_dim + args.multiple_of - 1) // args.multiple_of)

    self.w1 = nn.Linear(args.dim, hidden_dim, bias=False)
    self.w2 = nn.Linear(hidden_dim, args.dim, bias=False)
    self.w3 = nn.Linear(args.dim, hidden_dim, bias=False)

  def forward(self, x: torch.Tensor):
    swish = F.silu(self.w1(x))
    x_V = self.w3(x)
    x = swish * x_V
    x= self.w2(x)
    return x

class EncoderBlock(nn.Module):
  def __init__(self, args: ModelArgs):
    super().__init__()

    self.n_head=args.n_heads
    self.dim = args.dim
    self.head_dim = args.dim // args.n_heads

    self.attention = SelfAttention(args)
    self.feed_forward = FeedForward(args)

    # Normalization BEFORE the self attention
    self.attention_norm = RMSNorm(args.dim, eps=args.norm_eps)
    # Normalization BEFORE the feed forward block
    self.ffn_norm = RMSNorm(args.dim, eps= args.norm_eps)

  def forward(self, x: torch.Tensor, start_pos: int, freqs_complex: torch.Tensor):
    # (B, seq_len, Dim) + (B, seq_len, Dim) -> (B, seq_len, Dim)
    h = x + self.attention.forward(self.attention_norm(x), start_pos,freqs_complex)
    out = h + self.feed_forward.forward(self.ffn_norm(h))


class Transformer(nn.Module):

  def __init__(self, args: ModelArgs) -> None:
    super().__init__()
    assert args.vocab_size != -1, "Vocab size must be set"

    self.args = args
    self.vocab_size = args.vocab_size
    self.n_layers = args.n_layers
    self.tok_embeddings = nn.Embedding(self.vocab_size, args.dim)

    self.layers = nn.ModuleList()
    for _ in range(args.n_layers):
      self.layers.append(EncoderBlock(args))

    self.norm = RMSNorm(args.dim, eps=args.norm_eps)
    self.output = nn.Linear(args.dim, self.vocab_size, bias=False)

    self.freqs_complex = precompute_theta_pos_frequencies(self.args.dim // self.args.n_heads, self.args.max_seq_len * 2, device=self.args.device)

  def forward(self, tokens: torch.Tensor, start_pos: int):
    # (B, seq_len)
    batch_size, seq_len = tokens.shape
    assert seq_len == 1, "only one token at a time can be processed" #this only works for inference for training you must process more than 1 token at a time and must also remove KV cache

    # (B, Seq_Len) -> (B, seq_len, Dim)
    h = self.tok_embeddings(tokens)

    # Retrieve the pairs (m, theta) corresponding to the positions [start_pos, start_pos + seq_len]
    freqs_complex = self.freqs_complex[start_pos:start_pos+seq_len]

    # Consecutively apply all the encoder layers
    for layer in self.layers:
      h = layer(h, start_pos, freqs_complex)
    h = self.norm(h)

    output = self.output(h).float()
    return output


In [ ]:
from typing import Optional
import torch
import time
from pathlib import Path
import json
from sentencepiece import SentencePieceProcessor
from tqdm import tqdm

#from model import ModelArgs, Transformer

class LLaMA:

  def __init__(self, model: Transformer, tokenizer: SentencePieceProcessor, model_args: ModelArgs):
    self.model = model
    self.tokenizer = tokenizer
    self.args = model_args

  @staticmethod
  def build(checkpoints_dir: str, tokenizer_path: str, load_model: bool, max_seq_len: int, max_batch_size: int, device: str):
    prev_time = time.time()
    if load_model:
      checkpoints = sorted(Path(checkpoints_dir).glob('*.pth'))
      assert len(checkpoints) > 0, "No checkpoints files found"
      chk_path = checkpoints[0]
      print(f"Loading checkpoint {chk_path}")
      checkpoint = torch.load(chk_path, map_location="cpu")
      print(f"Loaded checkpoint in {(time.time() - prev_time):.2f}s")
      prev_time = time.time()

    with open (Path(checkpoints_dir) / "params.json", "r") as f:
      params = json.loads(f.read())

    model_args: ModelArgs = ModelArgs(
      max_seq_len=max_seq_len,
      max_batch_size=max_batch_size,
      device=device,
      **params
    )

    tokenizer = SentencePieceProcessor()
    tokenizer.load(tokenizer_path)
    model_args.vocab_size = tokenizer.vocab_size()

    if device == "cuda":
      torch.set_default_tensor_type(torch.cuda.HalfTensor)
    else:
      torch.set_default_tensor_type(torch.BFloat16Tensor)

    model = Transformer(model_args).to(device)
    if load_model:
      del checkpoint["rope.freqs"]
      model.load_state_dict(checkpoint, strict=True)
      print(f"loaded state dict in {(time.time() - prev_time):.2f}s")

    return LLaMA(model, tokenizer, model_args)

    def text_completion(self, prompts: list[str], temperature: float = 0.6, top_p: float = 0.9, max_gen_len: Optional[int] = None):
      if max_gen_len is None:
        max_gen_len = self.args.max_seq_len - 1
      # Convert each prompt into tokens
      prompt_tokens = [self.tokenizer.encode(prompt, out_type=int, add_bos=True, add_eos=False) for prompt in prompts]
      # Make sure the batch size is not too large
      batch_size = len(prompt_tokens)
      assert bath-size <= self.args.max_batch_size
      max_prompt_len = max(len(prompt) for prompt in prompt_tokens)
      # make sure the prompt length is not larger than the maximum seq length
      assert max_prompt_len <= self.args.max_seq_len
      total_len = min(self.args.max_seq_len, max_gen_len + max_prompt_len)

      # Create  the list that will contain the generated tokens, along with the initial prompt tokens
      pad_id = self.tokenizer.pad_id()
      tokens = torch.full((batch_size, total_len), pad_id, dtype = torch.long, device=device)
      for k, t in enumerate(prompt_tokens):
        # Populate the initial tokens with the prompt token, False otherwise
        tokens[k, :len(t)] = torch.tensor(t, dtype=torch.long, device=device)

      eos_reached = torch.Tensor([False] * batch_size, device=device)

In [ ]:
torch.manual_seed(42)

allow_cuda = False
device = "cuda" if torch.cuda.is_available() and allow_cuda else "cpu"

model = LLaMA.build(
    checkpoints_dir="llama-2-7b-chat",
    tokenizer_path="./tokenizer.model",
    load_model=True,
    max_seq_len=1024,
    max_batch_size=3,
    device=device
)
print("All ok")


Loading checkpoint llama-2-7b-chat/consolidated.00.pth
Loaded checkpoint in 5.32s


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:431.)
  _C._set_default_tensor_type(t)


loaded state dict in 46.14s
All ok
